## 상권 데이터_대분류 수집

In [1]:
import requests
import pandas as pd

In [2]:
Service_key = 'LVR82OMbkY0s4ySfvPsAAzVklADnEDLtwD4dgRH/T434bdVKyQhzWZew+9q669ncqTMgLzDna1iE7u4OTUKPGg=='

In [3]:
url = 'http://apis.data.go.kr/B553077/api/open/sdsc2/storeListInUpjong'
payload = dict(divId='indsLclsCd', key='I2', numOfRows=1000, pageNo=1, type='json',servicekey=Service_key)
r = requests.get(url, params=payload)

print(r.url)
print(r.status_code)
data = r.json()
data

n_data = data['body']['totalCount']
print(n_data)

total_pages = (int(data['body']['totalCount'])//1000)+1
print(total_pages)


http://apis.data.go.kr/B553077/api/open/sdsc2/storeListInUpjong?divId=indsLclsCd&key=I2&numOfRows=1000&pageNo=1&type=json&servicekey=LVR82OMbkY0s4ySfvPsAAzVklADnEDLtwD4dgRH%2FT434bdVKyQhzWZew%2B9q669ncqTMgLzDna1iE7u4OTUKPGg%3D%3D
200
869601
870


In [20]:
# header의 columns와 zip해서 나타내기
data.keys()
# data['body']['items']
# data['header']['columns']

dict_keys(['header', 'body'])

In [4]:
def json_unpack(data):
    result = {}
    header_columns = data[0]['header']['columns']
    for page_data in data:
        for dicts in page_data['body']['items']:
        #     print(dicts)
            for key, value in dicts.items():
        #         print(key, value)
                result.setdefault(key, []).append(value)
        # print(result)
    df = pd.DataFrame(result)
    # 컬럼명 변경 가능
    df.columns = header_columns
    return df

In [5]:
print(n_data)
print(total_pages)

676616
677


In [13]:
import time

def public_small_business(category):
    result_json = []
    page = 1
    while True:

        url = 'http://apis.data.go.kr/B553077/api/open/sdsc2/storeListInUpjong'
        Service_key = 'LVR82OMbkY0s4ySfvPsAAzVklADnEDLtwD4dgRH/T434bdVKyQhzWZew+9q669ncqTMgLzDna1iE7u4OTUKPGg=='

        payload = dict(divId='indsLclsCd', key=f"{category}", numOfRows=1000, pageNo=page, type='json',servicekey=Service_key)
        r = requests.get(url, params=payload)

        print(r.url, end='\r')
        print(r.status_code, end='\r')
        data = r.json()


        n_data = data['body']['totalCount']
        total_pages = (int(data['body']['totalCount'])//1000)+1
        result_json.append(data)
        time.sleep(0.1)

        if page <= total_pages:
            page += 1
        if page > 5:
            break 
    
    
    return result_json


In [107]:
G2_df = json_unpack(result_json)
# G2_df

In [29]:
major = ['G2','I1','I2','L1','M1','L1','P1','Q1','R1','S2']

for category in major:
#     print(category)
    result_by_category = public_small_business(category)
    df_by_category = json_unpack(result_by_category)
display(df_by_category)

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,MA010120220800029730,명품세탁소,,S2,수리·개인,S209,세탁,S20901,세탁소,S96912,...,1111018700100630009021108,명칭없음,서울특별시 종로구 통일로 266,110814,03030,,1,,126.956790,37.575882
1,MA010120220800079247,샵식스,,S2,수리·개인,S207,이용·미용,S20701,미용실,S96112,...,1135010600105870000010844,양지마을대림1단지,서울특별시 노원구 덕릉로71길 30,139923,01709,,1,,127.069781,37.650940
2,MA010120220800032079,수래쉬,,S2,수리·개인,S207,이용·미용,S20702,피부 관리실,S96113,...,1174010100102570000018778,고덕주공아파트,서울특별시 강동구 상암로 251,134788,05272,,1,,127.150688,37.548593
3,MA010120220800079790,정오디오,,S2,수리·개인,S205,가전제품 수리,S20501,가전제품 수리업,S95310,...,2635010500109380000003247,롯데,부산광역시 해운대구 해운대로483번길 10,612742,48065,,2,,129.147618,35.163441
4,MA010120220800079817,서울소독경기지사,,S2,수리·개인,S205,가전제품 수리,S20501,가전제품 수리업,S95310,...,4113310100124930000048843,그린빌,경기도 성남시 중원구 광명로 108,462824,13362,,3,,127.140665,37.435664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,MA010120220800382911,남자끼리,,S2,수리·개인,S207,이용·미용,S20701,미용실,S96112,...,2826011200101220002008026,현대아파트,인천광역시 서구 원적로 91,404809,22809,,,,126.681390,37.495280
4996,MA010120220800386882,들레뷰티,,S2,수리·개인,S207,이용·미용,S20702,피부 관리실,S96113,...,4825011900101690008086838,,경상남도 김해시 인제로230번길 50-11,621909,50813,,1,,128.906750,35.246913
4997,MA010120220800386905,마크1021호,사동점,S2,수리·개인,S207,이용·미용,S20701,미용실,S96112,...,4729012600102640000000001,,경상북도 경산시 솔숲길4길 2-8,712190,38603,,,,128.752816,35.807198
4998,MA010120220800386941,위니아24크린샵,전남대점,S2,수리·개인,S209,세탁,S20902,셀프 빨래방,S96912,...,2917010100102720006006463,,광주광역시 북구 중문로9번길 39,500873,61190,,,,126.916340,35.173883


## 상권 데이터 수집

In [33]:
def unpack_data(response):
    result = {}
    for item in response['body']['items']:
        for key, value in zip(response['header']['columns'], item.values()):
            result.setdefault(key, []).append(value)
    df = pd.DataFrame(result)
    return df

In [35]:
page = 1
result_dfs = []
while True:
    url = "http://apis.data.go.kr/B553077/api/open/sdsc2/storeListInUpjong"
    service_key = "LVR82OMbkY0s4ySfvPsAAzVklADnEDLtwD4dgRH/T434bdVKyQhzWZew+9q669ncqTMgLzDna1iE7u4OTUKPGg=="
    payload = dict(servicekey=service_key, divId="indsSclsCd", 
                  key="I21006", numOfRows=1000, pageNo=page, type="json")
    r = requests.get(url, params=payload)
    print(r.url, end="\r")
    print(r.status_code, end="\r")
    
    response = r.json()
    total_page = (response['body']['totalCount'] // 1000) + 1
    result_dfs.append(unpack_data(response))
    print(f"{page}/{total_page} 수집중", end="\r")
    if page < total_page:
        page += 1
    else:
        break
    time.sleep(0.2)
        
result = pd.concat(result_dfs)    
result = result.reset_index(drop=True)
result.to_csv("공공데이터_상권정보_202504.csv", encoding="utf-8", index=False)

In [37]:
# !pip install openpyxl
# conda install openpyxl

In [38]:
ori_data = pd.read_excel('./data/소상공인시장진흥공단_상가(상권)정보_업종분류(2302)_및_연계표_v1.0.xlsx', engine='openpyxl', header=[1])
ori_data

,대분류코드,대분류명,중분류코드,중분류명,소분류코드,소분류명
0,G2,소매업,G202,자동차 부품 및 내장품 소매업,G20201,타이어 소매업
1,G2,소매업,G202,자동차 부품 및 내장품 소매업,G20202,자동차 부품 소매업
2,G2,소매업,G203,모터사이클 및 부품 소매업,G20301,모터사이클 및 부품 소매업
3,G2,소매업,G204,종합 소매업,G20404,슈퍼마켓
4,G2,소매업,G204,종합 소매업,G20405,편의점
...,...,...,...,...,...,...
242,S2,수리 및 개인 서비스업,S209,세탁업,S20902,셀프 빨래방
243,S2,수리 및 개인 서비스업,S210,장례식장 및 관련 서비스업,S21001,장례식장
244,S2,수리 및 개인 서비스업,S210,장례식장 및 관련 서비스업,S21002,화장터/묘지/납골당
245,S2,수리 및 개인 서비스업,S211,기타 개인서비스,S21101,예식장업


In [36]:
# for code, name in zip(ori_data['소분류코드'], ori_data['소분류명']):
#     page = 1
#     result_dfs = []
#     while True:
#         url = "http://apis.data.go.kr/B553077/api/open/sdsc2/storeListInUpjong"
#         service_key = "8Ym5dhmVJdr12XzGnyYrQjSBS1QuRBYK8yHfx65JCl4vACM9uLKo8fxCVOFJkPB71llD7F2rOEROZnHgNGoj3A=="
#         payload = dict(servicekey=service_key, divId="indsSclsCd", 
#                       key=code, numOfRows=1000, pageNo=page, type="json")
#         r = requests.get(url, params=payload)
#         print(r.url, end="\r")
#         print(r.status_code, end="\r")

#         response = r.json()
#         total_page = (response['body']['totalCount'] // 1000) + 1
#         result_dfs.append(unpack_data(response))
#         print(f"{page}/{total_page} 수집중", end="\r")
#         if page < total_page:
#             page += 1
#         else:
#             break
#         time.sleep(0.2)

#     result = pd.concat(result_dfs)    
#     result = result.reset_index(drop=True)
#     result.to_csv(f"공공데이터_{name}_상권정보_202504.csv", encoding="utf-8", index=False)